# Difference-in-Difference (DiD)

- Basic DiD (2X2 with 사전평행)
- Control과 함께 DiD, DML을 이용한 DiD
- Staggered DiD, 2WFE
- DDD, Event Study
- 사전 평행 추세에 대한 통계적인 검정을 확인할 수 있는 파이썬 코드 필요. (matheus facure 책에서 쉽게 찾기 어려운 것 같습니다)

In [ ]:
import pandas as pd

# comment

### 질문이나 의견을 남겨주세요.
<script src="https://utteranc.es/client.js"
        repo="CausalInferenceLab/awesome-causal-inference-python"
        issue-term="pathname"
        theme="github-light"
        crossorigin="anonymous"
        async>
</script>